In [ ]:
import sys
import os
sys.path.append("../Calculator/")
from PDF import PDF
from OLEDPlot import OLEDNamedListPlot
from multiprocessing import Pool

In [ ]:
Voltage=1
Current=2
Current_density=3
Luminance=4
Radiance=5
Current_Efficacy=6
Luminous_Efficacy=7
EQE=8

In [ ]:
#obligatory
spectraPath=""
prefix=""
title=""
name=prefix
labelList=["", 
           "",
           "", 
           ""]
desiredPlots=[
    {
        "xCol":Voltage,
        "yCol":Current_density,
        "yCol2":Luminance
    },
    {
        "xCol":Current_density,
        "yCol":EQE,
        "custom":{
            "xLim":[10**-4,10],
            "xAxisLim":[10**-4,10]
        }
    }]
scaled=False

In [ ]:
#optional
customLims=True # better pick False running first Time
voltageLims=[None,[-2,10],[-2,10],[-2,10],[-2,10],[3,10],[3,10],[3,10],[3,10]]
yAxisLims=[None,[-2,10],None,None,[10**-2,10**5],[10**-2,10**5],[0,4],[0,15],[0,1]]
samples=4
inputParameters={
                    "spectraFile":spectraPath,
                    "alphaOffset":0,
                    "samples":samples,
                    "colorOffset":0,
                    "showErrorOnlyEvery":1,
                    "specBg":None,
                    "show":[[True,True]]*samples,
                    "errors":[[True,True]]*samples,
                    "customFontsize":[12,12,8,10,10],
                }
inputParametersForScaled=inputParameters.copy()
inputParametersForScaled.update({
                    "scaleX":0.47,
                    "customFontsize":[10,10,4,6,6],
                    "ax2Labels":False,
                    "titleBool":False
                    })

In [ ]:
def initPlot(xCol=1, showColTup=(2,3), customInputParameters=None):
    if customInputParameters is not None:
        inputParameters.update(customInputParameters)
        inputParametersForScaled.update(customInputParameters)
    scPlot=None
    if customLims:
        plot=OLEDNamedListPlot(prefix,
                               labelList,
                               name,
                               xCol=xCol,
                               showColTup=showColTup,
                               xLimOrig=voltageLims[showColTup[0]],
                               showColAxLim=yAxisLims, 
                               **inputParameters)
        if scaled:
            scPlot=OLEDNamedListPlot(prefix,
                                 labelList,
                                 name,
                                 xCol=xCol,
                                 showColTup=showColTup,
                                 xLimOrig=voltageLims[showColTup[0]],
                                 showColAxLim=yAxisLims, 
                                 **inputParametersForScaled)
    else:
        plot=OLEDNamedListPlot(prefix,
                               labelList,
                               name,
                               xCol=xCol,
                               showColTup=showColTup,
                               **inputParameters)
        if scaled:
            scPlot=OLEDNamedListPlot(prefix,
                                 labelList,
                                 name,
                                 xCol=xCol,
                                 showColTup=showColTup, 
                                 **inputParametersForScaled)
    if scPlot is None:
        return [plot]
    return [plot,scPlot]

In [ ]:
def buildPlotList(desiredPlot):
    try:
        yCol2=desiredPlot["yCol2"]
    except KeyError:
        yCol2=0
    try:
        cusPara=desiredPlot["custom"]
    except KeyError:
        cusPara=None
    if desiredPlot["yCol"]==0 or desiredPlot["xCol"]==0:
        raise
    return initPlot(xCol=desiredPlot["xCol"], showColTup=(desiredPlot["yCol"],yCol2), customInputParameters=cusPara)

In [ ]:
def processPlotPair(plotpair):
    return [plot.doPlot() for plot in plotpair]

In [ ]:
#multithreaded
pool = Pool(os.cpu_count())
plotList=pool.map(buildPlotList,desiredPlots)
multiOutput=pool.map(processPlotPair,plotList)
pool.close()

In [ ]:
plots=[[plotOutput[0] for plotOutput in plotPair] for plotPair in multiOutput]
files=[[plotOutput[1] for plotOutput in plotPair] for plotPair in multiOutput]

In [ ]:
PDF(files[0][0], size=(700,700*3/4)) #UI

In [ ]:
PDF(files[1][0], size=(700,700*3/4)) #EQE